In [1]:
# run this if you dont have triangle installed
# pip install triangle

# if you are working in a colab notebook
#!pip install triangle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.6 MB/s eta 0:00:00


In [4]:
import triangle as tr
import scipy.sparse
import numpy as np
import scipy.sparse.linalg
import matplotlib.pyplot as plt

In [ ]:
# Solve - div (k grad u ) + qu = f on a 2D domain with
# u = g on the boundary of the domain

# Specify a mesh size
mesh_sizes = [1/10,1/20,1/40]

# Pick some functions k, u, q, f, and g that satisfy the PDE in order to test
# your code
# EDIT THE FOLLOWING
def u(x):
    return 0
def grad_u(x):
    return []
def k(x):
    return 0
def q(x):
    return 0
def g(x):
    return 0
def f(x):
    return 0

In [ ]:
# Define the vertices of your domain.
# EDIT THIS
domain_vertices = [[]]

# Define the segments that connect the vertices of your domain.
# EDIT THIS
segments = [[]]

# Implement a quadrature rule of the form
# integral_K f(x) dx = sum_n w_n f(x_n), where
# K is the reference triangle with vertices [0,0], [1,0], and [0,1].
# You can find tables of quadrature points and weights for the reference
# triangle online.
# There are different conventions for how the quadrature weights wn are defined.
# With the convention we are using, the weights should sum to area(K) = 1/2.
# If the weights you find sum to 1 instead, just divide each weight by 2.
# EDIT THE FOLLOWING
quadrature_points = [[]]
quadrature_weights = []
number_quadrature_points = len(quadrature_points)

# Define the basis functions on the reference triangle as well as their
# gradients.
# EDIT THE FOLLOWING
def basis0(x): return 0
def basis1(x): return 0
def basis2(x): return 0
bases = [basis0, basis1, basis2]
grad_basis0 = [0,0]
grad_basis1 = [0,0]
grad_basis2 = [0,0]
grad_bases = [grad_basis0, grad_basis1, grad_basis2]

# Let's evaluate our basis functions at the quadrature points. We will need these later.
bases_quad_points = [[phi(x) for x in quadrature_points] for phi in bases]

L2_Error = np.zeros(len(mesh_sizes))
H1_semi_Error = np.zeros(len(mesh_sizes))

In [ ]:
def Construct_and_solve(mesh_sizes):
    m = 0
    for mesh_size in mesh_sizes:
        # Let's create our meshes.
        tri = {"vertices": domain_vertices, "segments": segments}
        mesh = (tr.triangulate(tri, "pqa{}".format(mesh_size ** 2)))

        # Let's plot the mesh.
        plt.figure()
        ax = plt.axes()
        tr.plot(ax, **mesh)

        # The elements are represented by a list of indices. The indices refer to
        # which vertices in the vertices list make up the three vertices of our
        # triangular element.
        elements = mesh["triangles"]
        vertices = mesh["vertices"]

        # We have 1 degree of freedom per vertex.
        number_vertices = len(vertices)
        global_matrix = scipy.sparse.lil_matrix((number_vertices, number_vertices))
        load_vector = np.zeros(number_vertices)

        # Now let's assemble our system.
        for element in elements:
            # Let's get the local element vertices.
            v0 = vertices[element[0]]
            v1 = vertices[element[1]]
            v2 = vertices[element[2]]

            # Let's compute all the relevant affine map information needed for our calculations.
            B = np.array([v1-v0, v2-v0]).transpose()
            detB = abs(np.linalg.det(B))
            invB = np.linalg.inv(B)

            # Now let's compute the local entries and add them to the global system.
            for i in range(0, 3):
                row = element[i]

                # Compute the local load vector contribution via quadrature.
                for n in range(0, number_quadrature_points):
                    xn = quadrature_points[n]
                    wn = quadrature_weights[n]

                    # Compute the local quadrature point by mapping it from the
                    # reference element
                    # EDIT THIS
                    Txn = 0

                    phi_in = bases_quad_points[i][n]
                    # Compute the local contribution to the load vector:
                    # EDI THIS
                    load_vector[row] += 0

                for j in range(0, 3):
                    col = element[j]

                    # Compute the local matrix contribution via quadrature.
                    for n in range(0, number_quadrature_points):
                        xn = quadrature_points[n]
                        wn = quadrature_weights[n]

                        # Compute the local quadrature point
                        # EDIT THIS
                        Txn = 0

                        # Compute the stiffness term
                        # EDIT THIS
                        grad_phi_i = np.matmul(grad_bases[i], invB)
                        grad_phi_j = np.matmul(grad_bases[j], invB)
                        dot_ij = np.dot(grad_phi_i, grad_phi_j)
                        stiff = 0

                        # Compute the mass term
                        # EDIT THIS
                        phi_in = bases_quad_points[i][n]
                        phi_jn = bases_quad_points[j][n]
                        mass = 0

                        # EDIT THIS
                        global_matrix[row, col] += 0

        # Now let's apply the Dirichlet boundary conditions.
        for i in range(len(mesh['vertices'])):
            # dof i is on the boundary if vertex_markers[i] == 1
            if mesh['vertex_markers'][i] == 1:
                # EDIT
                v = mesh['vertices'][i]
                global_matrix[0,0] = 0
                load_vector[0] = 0

        # Solve
        uh = scipy.sparse.linalg.spsolve(global_matrix.tocsr(), load_vector)

        # Compute the L2 error on the solution and the L2 error on the gradient of the solution via quadrature.
        L2error = 0
        H1semi_error = 0
        for element in elements:
            # Get local vertices
            v0 = vertices[element[0]]
            v1 = vertices[element[1]]
            v2 = vertices[element[2]]

            # Get coefficients of numerical solution on the local element
            uh0 = uh[element[0]]
            uh1 = uh[element[1]]
            uh2 = uh[element[2]]

            # Compute relevant affine map information
            B = np.array([v1-v0, v2-v0]).transpose()
            detB = abs(np.linalg.det(B))
            invB = np.linalg.inv(B)

            # Compute gradient of numerical solution on local element
            grad_phi_0 = np.matmul(grad_bases[0], invB)
            grad_phi_1 = np.matmul(grad_bases[1], invB)
            grad_phi_2 = np.matmul(grad_bases[2], invB)
            grad_uh = 0 # EDIT THIS

            for n in range(0, number_quadrature_points):
                xn = quadrature_points[n]
                wn = quadrature_weights[n]

                # Compute the local quadrature point
                Txn = 0  # EDIT THIS

                # Compute the exact solution at the quadrature point
                un = u(Txn)
                grad_un = grad_u(Txn)

                # Compute the H1 semi error
                dot_diff = np.dot(grad_uh - grad_un, grad_uh - grad_un)
                H1semi_error += 0

                # Compute the numerical solution at the quadrature point
                phi_0n = bases_quad_points[0][n]
                phi_1n = bases_quad_points[1][n]
                phi_2n = bases_quad_points[2][n]
                uhn = 0 # EDIT THIS

                dot_diff_L2 = np.dot(uhn-un,uhn-un)

                L2error += 0 # EDIT THIS

        L2_Error[m] = np.sqrt(L2error)
        H1_semi_Error[m] = np.sqrt(H1semi_error)
        m +=1

        # Let's display the error to the screen
        print("h:", mesh_size, "L2:", np.sqrt(L2error), "H1:", np.sqrt(H1semi_error))

    # Let's plot the numerical solution on the most refined mesh
    plt.figure()
    ax = plt.axes(projection="3d")
    xs = [v[0] for v in vertices]
    ys = [v[1] for v in vertices]
    ax.plot_trisurf(xs, ys, uh, triangles=elements, cmap="inferno")
    plt.show()


In [ ]:
def show_rate(mesh_sizes,errors,error_type):
    p = np.polyfit(np.log(mesh_sizes),np.log(errors),1)
    r = p[0]
    s = .75*errors[0]/N[0]**r
    print(f"{error_type} convergence rate: ", r)
    plt.figure()
    plt.loglog(mesh_sizes,errors, label = "Error Data")
    plt.loglog(mesh_sizes,s*mesh_sizes**r, label = "Poly Fit"),
    plt.xlabel("Max Area of Element")
    plt.ylabel(error_type)
    plt.title(f"{error_type} is O(mesh_size^{r})")
    plt.legend()

In [ ]:
Construct_and_solve(mesh_sizes)
show_rate(mesh_sizes,L2_Error,"L2 Errors")
show_rate(mesh_sizes,H1_semi_Error,"H1 Semi Errors")